# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
weather_df = pd.read_csv("../WeatherPy/Output/data/weather.csv", index_col="Unnamed: 0")
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Bredasdorp,13,ZA,1584205901,56,-34.53,20.04,69.80,17.22
1,Deputatsky,0,RU,1584205901,97,69.30,139.90,-34.26,1.07
2,Lompoc,90,US,1584205901,67,34.64,-120.46,57.99,6.93
3,Mossamedes,85,AO,1584205901,75,-15.20,12.15,76.44,7.67
4,Whitehorse,20,CA,1584205864,83,60.72,-135.05,1.00,1.90


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [230]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)
lat = weather_df['Lat']
long = weather_df['Lng']
coordinates = list(zip(long, lat))
locations = weather_df[['Lat', 'Lng']]
humidity = weather_df['Humidity']
max_humid = weather_df['Humidity'].max()

In [233]:
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=max_humid,
                                point_radius=1)

#Add layer

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [141]:
#Ideal Weather condition
    #Temp between 70 and 90 F
    #Wind speed less than 10 km an hour
    #humidity between 60 and 90
    
ideal_weather = weather_df.loc[(weather_df['Max Temp'] <= 90) & 
                               (weather_df['Max Temp'] >= 70) &
                               (weather_df['Wind Speed'] < 10) & 
                               (weather_df['Humidity'] <= 90) &
                               (weather_df['Humidity'] >= 70) &
                               (weather_df['Cloudiness'] < 50)]

ideal_weather = ideal_weather.reset_index(drop=True)
print(len(ideal_weather))
ideal_weather.head()

26


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Port Dickson,20,MY,1584205909,78,2.52,101.80,82.40,6.93
1,Port Hedland,7,AU,1584205943,88,-20.32,118.57,80.60,6.93
2,George Town,20,MY,1584205413,83,5.41,100.34,84.20,3.36
3,Kavaratti,18,IN,1584205977,71,10.57,72.64,82.38,9.60
4,Mahébourg,40,MU,1584205980,83,-20.41,57.70,78.80,3.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [208]:
hotel_df = ideal_weather[['Lat', 'Lng', 'City', 'Country']].copy()
#hotel_df = pd.DataFrame(columns=['City', 'Hotel Name'])
hotel_df['Hotel Name'] = ''
hotel_df['Hotel Lat'] = ''
hotel_df['Hotel Lng'] = ''
hotel_df['Hotel Address'] = ''
narrowed_city_df = pd.DataFrame()
hotel_df.head()

,Lat,Lng,City,Country,Hotel Name,Hotel Lat,Hotel Lng,Hotel Address
0,2.52,101.80,Port Dickson,MY,,,,
1,-20.32,118.57,Port Hedland,AU,,,,
2,5.41,100.34,George Town,MY,,,,
3,10.57,72.64,Kavaratti,IN,,,,
4,-20.41,57.70,Mahébourg,MU,,,,


In [209]:
# find the closest hotel of each type to coordinates

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get cooradinates from df
    location = f"{row['Lat']}, {row['Lng']}"

    # add keyword to params dict
    params['location'] = location
    
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']

    try:
        print(f"Closest hotel to {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        hotel_df.loc[index, 'Hotel Lat'] = results[0]['geometry']['location']['lat']
        
        hotel_df.loc[index, 'Hotel Lng'] = results[0]['geometry']['location']['lng']
        
        hotel_df.loc[index, 'Hotel Address'] = results[0]['vicinity']

        
    except:
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 0: Port Dickson.
Closest hotel to Port Dickson is Lexis Port Dickson.
------------
Retrieving Results for Index 1: Port Hedland.
Closest hotel to Port Hedland is The Esplanade Hotel.
------------
Retrieving Results for Index 2: George Town.
Closest hotel to George Town is Cititel Penang.
------------
Retrieving Results for Index 3: Kavaratti.
Closest hotel to Kavaratti is Hotels in Lakshadweep Islands.
------------
Retrieving Results for Index 4: Mahébourg.
Closest hotel to Mahébourg is Shandrani Beachcomber Resort & Spa.
------------
Retrieving Results for Index 5: Marawi.
Closest hotel to Marawi is Derogongan Residence.
------------
Retrieving Results for Index 6: Lodja.
Missing field/result... skipping.
------------
Retrieving Results for Index 7: Bambous Virieux.
Closest hotel to Bambous Virieux is Casa Tia Villa.
------------
Retrieving Results for Index 8: Cap Malheureux.
Closest hotel to Cap Malheureux is Pereybere Hotel & Spa.
------------
Retrievin

In [214]:
nan_value = float("NaN")
hotel_df.replace('',nan_value, inplace = True)
hotel_df = hotel_df.dropna()
hotel_df.head()

,Lat,Lng,City,Country,Hotel Name,Hotel Lat,Hotel Lng,Hotel Address
0,2.52,101.80,Port Dickson,MY,Lexis Port Dickson,2.550638,101.797059,"Batu 2, Jalan Seremban, Port Dickson"
1,-20.32,118.57,Port Hedland,AU,The Esplanade Hotel,-20.314012,118.576470,"2 Anderson Street, Port Hedland"
2,5.41,100.34,George Town,MY,Cititel Penang,5.421232,100.333104,"66, Jalan Penang, George Town"
3,10.57,72.64,Kavaratti,IN,Hotels in Lakshadweep Islands,10.576452,72.638399,Kavaratti
4,-20.41,57.70,Mahébourg,MU,Shandrani Beachcomber Resort & Spa,-20.448701,57.706641,Mahebourg
5,8.00,124.29,Marawi,PH,Derogongan Residence,8.005733,124.291998,"Sultan Omar Dianalan Boulevard, Marawi City"
7,-20.34,57.76,Bambous Virieux,MU,Casa Tia Villa,-20.349928,57.741995,"Domaine du Chausser Road, Anse Joncee, Grand Port"
8,-19.98,57.61,Cap Malheureux,MU,Pereybere Hotel & Spa,-19.994261,57.591778,Grand Baie
9,-20.52,57.52,Souillac,MU,Bils' Villa,-20.522379,57.528397,"293, Jean Pierre des Verney Gris Gris Souillac"
10,6.64,122.27,Tuburan,PH,Bohe Puno,6.621089,122.299215,Hadji Mohammed Ajul


In [236]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Hotel Lat", "Hotel Lng"]]

In [239]:
# Add marker layer ontop of heat map

# Create a map using state centroid coordinates to set markers
marker_locations = locations

# Create a marker_layer using the poverty list to fill the info box
markers = gmaps.marker_layer(marker_locations,
    info_box_content=hotel_info)
    
fig.add_layer(heat_layer)    
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))